In [1]:
import csv, random

# SYNTH_METHOD = 'ud'
# SYNTH_METHOD = 'UD'
# SYNTH_METHOD = 'arud+literal'
SYNTH_METHOD = 'ud+literal'
# SYNTH_METHOD = 'catib6'
# SYNTH_METHOD = 'pattern'
# SYNTH_METHOD = 'reinflect'

corpus_txfm = {}
with open(f'final_corpus_{SYNTH_METHOD}.csv', 'r', encoding='utf-8') as feat:
    reader = csv.reader(feat)
    for row in reader:
        corpus_txfm[row[0]] = row[1]

print(f"{len(corpus_txfm):,} sentences.")

corpus_txfm = list(corpus_txfm.items())
random.shuffle(corpus_txfm)
corpus_txfm = {k: v for k, v in corpus_txfm}

list(corpus_txfm.items())[:10]

33,564 sentences.


[('ظَلَفَ نَفْسَهُ عَمَّا يَشينُها', 'N NR عمR VR'),
 ('الْمُقْتَرِعُ فِي الانْتِخَابِ', 'N في N'),
 ('مَالَهُ صَادِرٌ وَلاَ وَارِدٌ', 'NR J ولا J'),
 ('أَيُّهَا النَّائِمُونَ اِسْتَيْقِظُوا', 'أيها J V'),
 ('بَدَأَ لَوْحُ البَابِ يَتَشَظَّى', 'V V N V'),
 ('تَرَكَ أَمْرَهُ نَدَباً', 'V NR V'),
 ('ضَمِنَ عَلَى أَهْلِهِ', 'N على NR'),
 ('اِجْتَمَعَ أعْيانُ القَوْمِ فِي القَصْرِ البَلَدِيِّ', 'V N N في N J'),
 ('اِنْتَظَرَ آخِرَ لَحْظَةٍ لِيَشْتَرِيَ خَرُوفَ الْعِيدِ', 'V J N لV N N'),
 ('يَسْتَمْتِعُ بِالسِّباحَةِ عَلى الشَّاطِئِ', 'V بN على N')]

# Remove Semantic Content

In [2]:
import pandas as pd

# stats = []
corpus = []
with open('ghani.txt', 'r', encoding='utf-8') as feat:
    for i, line in enumerate(feat):
        line = line.strip()[1:-1]
        # if len(line) == 0: continue
        # toks = disamb.disambiguate(simple_word_tokenize(line))
        # tok_count = 0
        # poss = set()
        # roots = set()
        # for t in toks:
        #     tok_count += 1
        #     if len(t.analyses) == 0:
        #         continue
        #     tok_count += t.analyses[0].analysis['pos'].count('+')
        #     roots.add(t.analyses[0].analysis['root'])
        #     for pos in t.analyses[0].analysis['pos'].split('+'): poss.add(pos)

        # if tok_count < 2: continue
        # stats.append({
        #     'chars': len(line),
        #     'words': line.count(' ') + 1,
        #     'tokens': tok_count,
        #     'roots': len(roots),
        #     'poss': len(poss),
        # })
        corpus.append(line)


corpus[910:915]

['أعْلَنَتِ الشَّرِكَةُ عَنْ بِضَاعَتِهَا بِوَاسِطَةِ إِعْلاَناتٍ تِجَارِيَّةٍ',
 'أُنَاشِدُكُمْ أَنْ نَعْمَلَ كَأُمَّةٍ جُمِعَتْ كَلِمَتُهَا وَوُحِّدَتْ غَايَتُهَا',
 'إِنَّ اللَّهَ لاَ يُغَيِّرُ مَا بِقَوْمٍ حَتَّى يُغَيِّرُوا مَا بِأَنْفُسِهِمْ',
 'إِنَّ النّبِيَّ تَوَضَّأَ فَضَاقَ عَنْ يَدَيْهِ كُمَّا جِمَازَةٍ كَانَتْ عَلَيْه',
 'إِنْ كَانَ لَسِناً سُمِّيَ مِهْذَاراً وإِنْ كَانَ صَمُوتاً سُمِّيَ عَيِيّاً']

In [3]:
# pd.DataFrame(stats).describe()

In [4]:
# pd.DataFrame(stats).sum()

In [2]:
from tqdm import tqdm
import csv
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.reinflector import Reinflector
# from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer
from camel_tools.tagger.default import DefaultTagger
from camel_tools.tokenizers.word import simple_word_tokenize

if 'db' not in locals(): db = MorphologyDB.builtin_db(flags='rag')
# TODO use bert instead
if 'disamb' not in locals(): disamb = MLEDisambiguator.pretrained()
# if 'disamb' not in locals(): disamb = BERTUnfactoredDisambiguator.pretrained(model_name='msa')

reinflector = Reinflector(db)
tokenizer = MorphologicalTokenizer(disambiguator=disamb, scheme='atbseg')
tagger = DefaultTagger(disamb, 'pos')

dbg_sent = disamb.disambiguate(simple_word_tokenize('صدقني لا أموال عندي فأبذله إليكم'))
dbg_sent = disamb.disambiguate(simple_word_tokenize('أوعر الطريق على المسافر'))
print(f"DISAMBIG={dbg_sent}")

# 'lex_logprob', 'pos_logprob', 'pos_lex_logprob', 'd3seg', atbtok', 
# 'd3tok', 'stt', 'bw', 'd1tok', 'atbseg', 'd2seg', 'd2tok', 'bwtok', 'd1seg'
# 'source', 'gloss', 'diac', 'lex', 'caphi'
# 'pos', 'ud', 'catib6',
# 'pattern', 'root',  
# 'enc0', 'prc0', 'prc1', 'prc2', 'prc3', 
# 'asp', 'num', 'cas', 'form_num', 'per', 'gen', 'form_gen', 'mod', 'vox', 'rat', 
for fs in [
    ['pos', 'catib6', 'ud'],
    ['root', 'pattern'],
    ['enc0', 'prc0', 'prc1', 'prc2', 'prc3'],
    ['form_gen', 'gen', 'form_num', 'num'],
    ['asp', 'cas', 'per', 'mod', 'vox', 'rat'],
]:
    print()
    print(fs)
    for w in dbg_sent:
        anal = w.analyses[0].analysis
        print(w.word, [anal[f] for f in fs])

print()
for w in dbg_sent:
    lemma = 'فعل'
    anal = w.analyses[0].analysis
    feats = {}
    for feat in anal.keys():
        if feat in ['ud', 'enc0', 'prc0', 'prc1', 'prc2', 'prc3']:
            # VERB = vox, asb
            # NOUN = per
            # rat, cas, pattern, form_gen, form_num, num, gen
        # if f in['stem', 'stemgloss', 'stemcat']: continue
            feats[feat] = anal[feat]
    r = reinflector.reinflect(lemma, feats)
    if len(r) == 0: print('???')
    else: print(r[0]['diac'])

DISAMBIG=[DisambiguatedWord(word='أوعر', analyses=[ScoredAnalysis(score=1.0, analysis={'diac': 'أَوْعَر', 'lex': 'أَوْعَر', 'bw': 'أَوْعَر/NOUN', 'gloss': 'rougher;roughest;more;most_rugged', 'pos': 'noun', 'prc3': '0', 'prc2': '0', 'prc1': '0', 'prc0': '0', 'per': 'na', 'asp': 'na', 'vox': 'na', 'mod': 'na', 'stt': 'i', 'cas': 'u', 'enc0': '0', 'rat': 'n', 'source': 'lex', 'form_gen': 'm', 'form_num': 's', 'd3seg': 'أَوْعَر', 'caphi': '2_a_w_3_a_r', 'd1tok': 'أَوْعَر', 'd2tok': 'أَوْعَر', 'pos_logprob': -2.411992, 'd3tok': 'أَوْعَر', 'd2seg': 'أَوْعَر', 'pos_lex_logprob': -99.0, 'num': 's', 'ud': 'ADJ', 'gen': 'm', 'catib6': 'NOM', 'root': '#.ع.ر', 'bwtok': 'أَوْعَر', 'pattern': 'أَوْ2َ3', 'lex_logprob': -99.0, 'atbtok': 'أَوْعَر', 'atbseg': 'أَوْعَر', 'd1seg': 'أَوْعَر', 'stem': 'أَوْعَر', 'stemgloss': 'rougher;roughest;more;most_rugged', 'stemcat': 'Nel'}, diac='أَوْعَر', pos_lex_logprob=-99.0, lex_logprob=-99.0)]), DisambiguatedWord(word='الطريق', analyses=[ScoredAnalysis(score=1.0

In [3]:
TAG2LETTER = {
    'PROPN': 'N',
    'NOUN': 'N',
    'PRON': 'N',
    'NUM': 'N',
    'DET': 'N5',
    'VERB': 'V1',
    'AUX': 'A1',
    'ADP': 'A2',
    'PART': 'A3',
    'SCONJ': 'A5',
    'CCONJ': 'A4',
    'ADJ': 'J1',
    'ADV': 'J2',
    'PUNCT': '.',
    'INTJ': 'W',
    'X': 'X',
}
LETTER2TAG = {k: v for v, k in TAG2LETTER.items()}
# assert(len(TAG2LETTER) == len(LETTER2TAG))

LETTER2TAG2 = {
    'العلم': 'PROPN',
    'اسم': 'NOUN',
    # 'هم': 'PRON',
    'قام': 'VERB',
    'كان': 'AUX',
    'نعت': 'ADJ',
    'حال': 'ADV',
    # 'L': 'ADP',
    # 'T': 'PART',
    # '.': 'PUNCT',
    'رقم': 'NUM',
    # 'S': 'SCONJ',
    # 'C': 'CCONJ',
    # 'D': 'DET',
    # 'I': 'INTJ',
    'مجهول': 'X',
}
TAG2LETTER2 = {k: v for v, k in LETTER2TAG2.items()}
assert(len(TAG2LETTER2) == len(LETTER2TAG2))

if SYNTH_METHOD == 'catib6':
    tagger = DefaultTagger(disamb, 'catib6')
else:
    tagger = DefaultTagger(disamb, 'ud')

def synth_token(token, tag):
    if SYNTH_METHOD in ['UD', 'catib6', 'pattern']:
        return tag
    if SYNTH_METHOD in ['ud+literal'] and tag in ['ADP', 'PART', 'SCONJ', 'CCONJ']:
        return token.replace('_', '')
    if SYNTH_METHOD == 'arud+literal':
        if tag in ['ADP', 'PART', 'SCONJ', 'CCONJ', 'PUNCT', 'DET', 'PRON', 'INTJ']:
            return token.replace('_', '')
        else:
            return TAG2LETTER2[tag]
    return TAG2LETTER[tag]

def synth_txt(txt, min_words=3):
    # ignore short setences since they don't tell us much
    if txt.count(' ') < min_words - 1: return None
    txt = simple_word_tokenize(txt)
    tokens = tokenizer.tokenize(txt)
    tags = tagger.tag(txt)
    if '' in tags:
        return None

    joiner = ' '
    if SYNTH_METHOD in ['UD', 'catib6']:
        joiner = '+'

    ans = []
    if SYNTH_METHOD == 'pattern':
        for token in disamb.disambiguate(txt):
            if len(token.analyses) == 0: return
            anal = token.analyses[0].analysis
            ans.append(anal['pattern'])
    else:
        for token_agg, tag_agg in zip(tokens, tags):
            if token_agg is None or tag_agg is None or token_agg.count('+') != tag_agg.count('+'):
                return None
            ans.append(joiner.join([synth_token(token, tag) for token, tag in zip(token_agg.split('+'), tag_agg.split('+'))]))
    return ' '.join(ans)

print(SYNTH_METHOD)

print(synth_txt("بالله", 1))
print(synth_txt("بما", 1))
print(synth_txt("مازال", 1))
print(synth_txt("ثلاثة", 1))
print(synth_txt("إليك عني يا أخي فإنه قد تكون عندي هموم يطيش لها عقل ذي الحلم فرقا", 1))
print(synth_txt("منحني الله الصحة والعافية ونصيبا وافرا من العلم و 10 أبناء، والحسود لا يسود", 1))
print(synth_txt("بلى! نظرت طويلا إلى هؤلاء الأطفال تحت بيتنا في حر الصيف", 1))

print(synth_txt("عمر الفتى ذكره لا طول مدته، وموته خزيه لا يومه الآتي", 1))
print(synth_txt("وما المرء إلا كالهلال وضوئه، وما المرء إلا الأصغران لسانه ومعقوله", 1))

ud+literal
None
None
None
None
إلي N عن N يا N N ف إن N A1 V1 N N N V1 ل N N N N N
V1 N N N و N و N J1 من N و N N . و J1 لا V1
ب N . V1 J1 إلى N5 N N N N في J1 N
N N V1 N لا N N N . و N N N N لا N N J1
و N N إلا ك N و N N . و N N إلا J1 N N و J1 N


In [7]:
corpus_txfm = {}
txfm_errors = []
for sent in tqdm(corpus):
    sent_txfm = synth_txt(sent)
    if sent_txfm is None: txfm_errors.append(sent)
    else: corpus_txfm[sent] = sent_txfm

print(SYNTH_METHOD)
print(f"{len(txfm_errors)}/{len(corpus)} FAILED")
list(corpus_txfm.items())[910:915]
# 5m8m 6m 2m2s

  0%|          | 0/70675 [00:00<?, ?it/s]

100%|██████████| 70675/70675 [04:26<00:00, 265.02it/s]  

arud+literal
37106/70675 FAILED


[('لاَ أَثِقُ بِحُكْمِ نَفْسِي حَتَّى يُؤَيِّدَ النَّاسُ ظَنِّي أَوْ يُكَذِّبُوهُ',
  'لا قام ب اسم اسم ي حتى قام اسم اسم ي أو قام ه'),
 ('لاَ يَجُوزُ القَبْضُ عَلَى إِنْسَانٍ أَوْ حَجْزُهُ أَوْ نَفْيُهُ تَعَسُّفاً',
  'لا قام اسم على اسم أو قام ه أو اسم ه اسم'),
 ('لاَ يَنَالُ الإنْسَانُ الغَايَةَ إلاَّ بِالجِدِّ وَسَهَرِ اللَّيَالِي',
  'لا قام اسم اسم إلا ب اسم و قام اسم'),
 ('لَقَدْ حَقَّقَتِ البَشَرِيَّةُ تَقَدُّماً هائِلاً في القَرْنِ العِشْرِينَ',
  'ل كان قام نعت اسم نعت في اسم اسم'),
 ('لَمْ يَكُنْ يَرَى فِي السُّهُولِ الَّتِي تَمُرُّ مِنْهَا إِلاَّ الْغَيْضَةَ',
  'لم قام قام في اسم التي قام من ها إلا اسم')]

In [8]:
print(SYNTH_METHOD)
with open(f'final_corpus_{SYNTH_METHOD}.csv', 'w', encoding='utf-8') as feat:
    writer = csv.writer(feat, quoting=csv.QUOTE_NONNUMERIC)
    for sent, txfm in corpus_txfm.items():
        writer.writerow([sent, txfm])

arud+literal


# Train Embedder

In [9]:
!pip install -Uqq sentence_transformers

from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import random

print(SYNTH_METHOD)
train_sentences = list(corpus_txfm.values())
train_sentences = random.sample(train_sentences, 1000)
print('\n'.join(train_sentences[:10]))

# model_name = "bert-base-uncased"
# model_name = "google-bert/bert-base-multilingual-cased"
# model_name = "distilbert/distilroberta-base"
# model_name = "CAMeL-Lab/bert-base-arabic-camelbert-ca"
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa-sixteenth"

word_embedding_model = models.Transformer(model_name)
embedding_len = word_embedding_model.get_word_embedding_dimension()
print(embedding_len)

pooling_model = models.Pooling(embedding_len, "cls")
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
train_loss = losses.DenoisingAutoEncoderLoss(
    model, decoder_name_or_path=model_name, tie_encoder_decoder=True
)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    weight_decay=0,
    scheduler="constantlr",
    optimizer_params={"lr": 3e-5},
    show_progress_bar=True,
    output_path=f'ar_nyuad-ud/final_{SYNTH_METHOD}',
    save_best_model=True,
)
# 1m30s

/home/msid/fastai/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


arud+literal
قام العلم اسم اسم عن اسم هم
قام في اسم
العلم اسم ب اسم
قام اسم ب اسم
قام في اسم نعت
اسم اسم أو اسم
قام اسم في اسم ه اسم و قام
اسم في اسم نعت
قام اسم اسم
قام اسم نعت ه ب اسم
768


When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa-sixteenth and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder

Step,Training Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

# Embed

In [4]:
print(SYNTH_METHOD)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(f"ar_nyuad-ud/final_{SYNTH_METHOD}")

ud+literal


/home/msid/fastai/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
import random

print(SYNTH_METHOD)
embeddings = model.encode(list(corpus_txfm.values())[:10_000])
print(embeddings.shape)
print('\n'.join(list(corpus_txfm.keys())[:10]))
model.similarity(embeddings[:10,:], embeddings[:10,:])
# 9s 13s

arud+literal
(10000, 768)
لِكُلِّ إِنْسانٍ الحَقُّ في الوُجودِ والحُرِّيَّةِ والأمْنِ، والحَقُّ في حُرِّيَّةِ الرَّأْيِ والتَّعْبيرِ بِدونِ تَمْيِيزٍ بِسَبَبِ العُنْصُرِ أَوِ اللَّوْنِ أَوِ الجِنْسِ أَوِ الثَّقافَةِ أَوِ الدِّينِ أَوِ الرَّأْيِ
طَائِرٌ مِنْ فَصيلَةِالوَزِّيَّاتِ، مِنْ رُتْبةِ الكَفِّيَّاتِ، لَهُ مِنْقارٌ عَريضٌ مُلَوَّنٌ، قَصيرُ العُنُقِ والرِّجْلَيْنِ، طَويلُ الأجْنِحَةِ، يَخْتَلِفُ عَنِ الإِوَزِّ، وَإِنْ كانَ يُشْبِهُهُ وَهُوَ طائِرٌ مائِيٌّ لَهُ قُدْرَةٌ على الطَّيَران
يَبْلُغُ عَدَدُ حُروفِ هِجاءِ اللُّغَةِ العَرَبِيَّةِ ثَمانِيَةً وَعِشْرينَ حَرْفاً هي: ا ب ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ك ل م ن هـ و ي. وَهُوَ ما يُسَمَّى بِحُروفِ الْمَبانِ
اِنْقَشَعَ السّحابُ» لا مَحَلَّ لها مِنَ الإِعْرابِ لأَنَّها جُمْلَةٌ اِبْتِدائِيَّةٌ، والجُمَلُ التي لها مَحَلٌّ مِنَ الإِعْرابِ، هي التي تَحَلُّ مَحَلَّ مُفْرَدٍ، أي ما لَيْس جُمْلَةً ولا شِبْهَ جُمْلَة
جَمَاعَةٌ مِنَ النَّاسِ تَجْمَعُهُمْ رَوَابِطُ تَارِيخِيَّةٌ مُشْتَرَكَةٌ، قَدْ يَكُونُ فِيهَا مَا هُوَ لُغَوِيٌّ أوْ

tensor([[1.0000, 0.8703, 0.9304, 0.8446, 0.8305, 0.9437, 0.9184, 0.8833, 0.8648,
         0.8001],
        [0.8703, 1.0000, 0.9276, 0.9123, 0.9417, 0.9337, 0.9602, 0.9475, 0.9488,
         0.9083],
        [0.9304, 0.9276, 1.0000, 0.8592, 0.8970, 0.9576, 0.9403, 0.9088, 0.9303,
         0.8328],
        [0.8446, 0.9123, 0.8592, 1.0000, 0.8382, 0.8589, 0.9131, 0.9260, 0.8600,
         0.9531],
        [0.8305, 0.9417, 0.8970, 0.8382, 1.0000, 0.9161, 0.9224, 0.9078, 0.9566,
         0.8534],
        [0.9437, 0.9337, 0.9576, 0.8589, 0.9161, 1.0000, 0.9554, 0.8899, 0.9464,
         0.8321],
        [0.9184, 0.9602, 0.9403, 0.9131, 0.9224, 0.9554, 1.0000, 0.9323, 0.9554,
         0.8868],
        [0.8833, 0.9475, 0.9088, 0.9260, 0.9078, 0.8899, 0.9323, 1.0000, 0.8960,
         0.9163],
        [0.8648, 0.9488, 0.9303, 0.8600, 0.9566, 0.9464, 0.9554, 0.8960, 1.0000,
         0.8417],
        [0.8001, 0.9083, 0.8328, 0.9531, 0.8534, 0.8321, 0.8868, 0.9163, 0.8417,
         1.0000]])

# Cluster

In [9]:
from sklearn.cluster import AffinityPropagation
affinity = AffinityPropagation(random_state=5).fit(embeddings)
# # 3m24s 4m10s 4m15s 3m50s 3m55s 3m55s 4m

In [10]:
from sklearn.cluster import Birch
birch = Birch(n_clusters=None).fit(embeddings)
# 8s 1s 1s 1s 2s

In [11]:
from sklearn.cluster import HDBSCAN
hdb = HDBSCAN(min_cluster_size=3, max_cluster_size=50, n_jobs=8).fit(embeddings)
# # 35s 32s 29s 30s 31s 35s 33s

In [12]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=int(len(embeddings)**0.5) * 8).fit(embeddings)
# # 3m4s 6s 2s 7s 9s 8s 13s 14s

In [13]:
from sklearn.cluster import MeanShift
meanshift = MeanShift(bandwidth=2).fit(embeddings)
# # 1m35s 1m22s 1m46s 1m40s 1m50s 1m

In [14]:
from sklearn.cluster import OPTICS
optics = OPTICS(min_samples=3).fit(embeddings)
# # 5m20s 2m14s 2m20s 2m20s

In [16]:
from collections import defaultdict

def save_clustering(algorithm, labels):
    clusters = defaultdict(lambda: [])
    for sent, label in zip(corpus2, labels):
        clusters[label].append(sent)
    
    if -1 not in clusters:
        clusters[-1] = []
    for ss in clusters.values():
        if len(ss) == 1:
            clusters[-1].extend(ss)

    clusters = {k: v for k, v in clusters.items() if len(v) > 1 or k == -1}
    clusters = {k: v for _, v, k in sorted([(len(v), sorted(v), k) for k, v in clusters.items()])}
    n_unclustered = len(clusters[-1])
    
    path = f'final_cluster_{SYNTH_METHOD}_{algorithm}.txt'
    print(path)
    with open(path, 'w') as file:
        file.write(f"CLUSTER_COUNT = {len(clusters)}\n")
        file.write(f"UNCLUSTERED = {n_unclustered:,} / {sum(map(len, clusters.values())):,}\n")
        for label, sents in clusters.items():
            file.write(f"\nCLUSTER {label} ({len(sents):,} sentences)\n")
            for sent in sents:
                if label == -1:
                    file.write(f" xx {sent}\n")
                else:
                    file.write(f"    {sent}\n")
    return clusters


print(SYNTH_METHOD)
save_clustering("affinity", affinity.labels_)
save_clustering("birch", birch.predict(embeddings))
save_clustering("hdbscan", hdb.labels_)
save_clustering("kmeans", kmeans.labels_)
save_clustering("meanshift", meanshift.labels_)
save_clustering("optics", optics.labels_)

ud+literal
final_cluster_ud+literal_affinity.txt
final_cluster_ud+literal_birch.txt
final_cluster_ud+literal_hdbscan.txt
final_cluster_ud+literal_kmeans.txt
final_cluster_ud+literal_meanshift.txt
final_cluster_ud+literal_optics.txt


{9: ['إِنْ كُنْتِ أَزْمَعْتِ الفِرَاقَ فَإِنَّمَا',
  'فَإِذَا شَرَبْتُ فَإِنَّنِي مُسْتَهْلِكٌ',
  'وَإِذَا ظُلِمْتُ فَإِنَّ ظُلْمِي بَاسِلٌ'],
 5: ['تَطِسُ الإِكَامَ بِوَخْذِ خُفٍّ مِيْثَمِ',
  'غَمَرَاتِهَا الأَبْطَالُ غَيْرَ تَغَمْغُمِ',
  'وَسْطَ الدِّيَارِ تَسُفُّ حَبَّ الخِمْخِمِ'],
 10: ['عَسِراً عَلَيَّ طِلَابُكِ ابْنَةَ مَخْرَمِ',
  'قَدْحَ المُكِبِّ عَلَى الزِّنَادِ الأَجْذَمِ',
  'لَيسَ الكَرِيمُ عَلَى القَنَا بِمُحَرَّمِ'],
 2: ['مَالِي وَعِرْضِي وَافِرٌ لَم يُكْلَمِ',
  'وَ مُدَجِجٍ كَرِهَ الكُمَاةُ نِزَالَهُ',
  'وَأَبِيتُ فَوْقَ سَرَاةِ أَدْهَمَ مُلْجَمِ'],
 7: ['حَشَّ الوَقُودُ بِهِ جَوَانِبَ قُمْقُمِ',
  'غَضْبَى اتَّقَاهَا بِاليَدَينِ وَبِالفَمِ',
  'هَزِجاً يَحُكُّ ذِرَاعَهُ بِذِرَاعِهِ',
  'يُحْذَى نِعَالَ السِّبْتِ لَيْسَ بِتَوْأَمِ'],
 4: ['سَحّاً وَتَسْكَاباً فَكُلَّ عَشِيِّةٍ',
  'فَتَجَسَّسِي أَخْبَارَهَا لِيَ وَاعْلَمِي',
  'فَتَرَكْتُهُ جَزَرَ السِّبَاعِ يَنُشْنَهُ',
  'فَتَرَكْنَ كُلَّ قَرَارَةٍ كَالدِّرْهَمِ'],
 6: ['صَعْلٍ يَعُودُ بِذِي العُشَيرَةِ بَيْضَ

# Application

In [19]:
print(SYNTH_METHOD)
print("SYNTACTIC SIMILARITY")


dbg_txt = """
لا يعرف كوعه من بوعه
""".strip()
print(dbg_txt)
dbg_txt = synth_txt(dbg_txt)
print(dbg_txt)
dbg_txt = model.encode([dbg_txt])
print(dbg_txt.shape)

import torch
sim = model.similarity(dbg_txt, embeddings)
print(sim.shape)
top = torch.topk(sim, 100)
top = [(i, v) for i, v in zip(top.indices[0], top.values[0]) if v.item() > .9]

for dbg_index, dbg_val in top:
    print()
    dbg_match = list(corpus_txfm.items())[dbg_index]
    print(dbg_val.item() * 1000000 // 100 / 100)
    print(dbg_match[0])
    print(dbg_match[1])

arud+literal
SYNTACTIC SIMILARITY
لا يعرف كوعه من بوعه
لا قام اسم ه من اسم ه
(1, 768)
torch.Size([1, 10000])

97.38
لاَ يَأتِيهِ البَاطِلُ مِنْ بَيْنَ يَدَيْهِ
لا قام ه اسم من اسم اسم ه

95.78
لَمْ يَنْسَحِبْ مِنْ عَمَلِهِ مُصَادَفَةً
لم قام من اسم ه اسم

95.34
لَمْ يَكُنْ مِنَ السَّهْلِ تَجْرِيدُهُ مِنَ السِّلاَحِ
لم قام من اسم اسم ه من اسم

95.31
لاَ يَعْرِف يَدَ الشَّيْءِ مِنْ رِجْلِهِ
لا قام اسم اسم من اسم ه

95.3
رَجُلٌ أَبْلَهُ لاَ يَعْرِفُ مَا يُخْرِجُ مِنْ أُمِّ دِمَاغِهِ
اسم اسم ه لا قام ما قام من أم اسم ه

94.68
أَخْرَجَ فَجأةً سِلاَحَهُ مِنْ مِعْطَفِهِ
قام ف اسم اسم ه من اسم ه

94.36
مِنْ كَثْرَةِ بُكائِهِ اِسْتَنْزَفَ دُموعَهُ
من اسم اسم ه قام اسم ه

94.29
مَا حَمَلَتْهُ العُرُوسُ مِنْ بَائِنَةٍ يَفُوقُ مَا أَخَذَتْهُ مِنْ صَدَاقٍ
ما اسم ه اسم من اسم ه قام ما قام ه من اسم

94.25
يُصيبُهُ الزَّعَلُ مِنْ عَمَلِهِ الرَّتيبِ
قام ه اسم من اسم ه اسم

94.09
كُنْتُ أَسْمَعُ شَخِيرَهُ مِنَ الغُرْفَةِ الْمُجَاوِرَةِ
قام قام اسم ه من اسم اسم

93.98
مَا زَالَ لَدَيْهِ مُتَّسَعٌ مِنَ ال

In [20]:
from sentence_transformers import SentenceTransformer

model_sem = SentenceTransformer("sentence-transformers/stsb-xlm-r-multilingual")

print("SEMANTIC SIMILARITY")
print(SYNTH_METHOD)
dbg_embeds_sem = model_sem.encode(dbg_txt)
print(dbg_embeds_sem.shape)
sim_sem = model_sem.similarity(dbg_embeds_sem, dbg_embeds_sem)
sim_sem

SEMANTIC SIMILARITY
arud+literal


TypeError: object of type 'numpy.float32' has no len()

In [21]:
for i, j in [
    ('عمر الفتى ذكره لا طول مدته', 'وموته خزيه لا يومه الآتي'),
    ('وما المال والأهلون إلا وديعة', 'ولا بد يوما أن ترد الودائع'),
    ('وما المرء إلا كالهلال وضوئه', 'وما المال والأهلون إلا وديعة'),
    ('أغرقت الأمطار أراضي واسعة', 'أصدرت الوزارة بيانا جديدا'),
    ('عمر الفتى ذكره لا طول مدته', 'أن يبقى ذكر الفتى خير من أن يعمر طويلا',),
    ('عمر الفتى ذكره لا طول مدته', 'يعمر الفتى ما دام ذكره'),
    ('عمر الفتى ذكره لا طول مدته', ' يطول عمر الفتى ليس بطول السنين لكن الذكر'),
]:
    enc_sem = model_sem.encode([i, j])
    synth = list(map(synth_txt, [i, j]))
    print(synth)
    enc = model.encode(synth)
    print(f"{i} <=> {j}")
    print(f"  meaning = {model_sem.similarity(enc_sem, enc_sem)[0][1] * 100:.1f}%")
    print(f"  syntax  = {model.similarity(enc, enc)[0][1] * 100:.1f}%")

['العلم اسم قام ه لا اسم اسم ه', 'و اسم ه اسم ه لا اسم ه نعت']
عمر الفتى ذكره لا طول مدته <=> وموته خزيه لا يومه الآتي
  meaning = 33.3%
  syntax  = 87.1%
['و ما اسم و اسم إلا اسم ه', 'و لا اسم اسم أن قام اسم']
وما المال والأهلون إلا وديعة <=> ولا بد يوما أن ترد الودائع
  meaning = 33.5%
  syntax  = 88.3%
['و ما اسم إلا ك العلم و اسم ه', 'و ما اسم و اسم إلا اسم ه']
وما المرء إلا كالهلال وضوئه <=> وما المال والأهلون إلا وديعة
  meaning = 58.6%
  syntax  = 91.2%
['قام اسم اسم نعت', 'قام اسم اسم نعت']
أغرقت الأمطار أراضي واسعة <=> أصدرت الوزارة بيانا جديدا
  meaning = 19.3%
  syntax  = 100.0%
['العلم اسم قام ه لا اسم اسم ه', 'أن قام قام اسم اسم من أن قام نعت']
عمر الفتى ذكره لا طول مدته <=> أن يبقى ذكر الفتى خير من أن يعمر طويلا
  meaning = 73.0%
  syntax  = 71.0%
['العلم اسم قام ه لا اسم اسم ه', 'قام اسم ما قام قام ه']
عمر الفتى ذكره لا طول مدته <=> يعمر الفتى ما دام ذكره
  meaning = 92.8%
  syntax  = 76.9%
['العلم اسم قام ه لا اسم اسم ه', 'قام العلم اسم قام ب اسم اسم لكن اسم']
عمر الفتى

In [22]:
dbg_txt = [
    'عمر الفتى ذكره لا طول مدته',
    'وموته خزيه لا يومه الآتي',
    'أن يبقى ذكر الفتى خير من أن يعمر طويلا',
    'ولا بد يوما أن ترد الودائع',
    'وما المرء إلا كالهلال وضوئه',
    'وما المال والأهلون إلا وديعة',
"""
ليس كل ما يتمنى المرء يدركه
""".strip(),
]

print(SYNTH_METHOD)
print("SYNTACTIC SIMILARITY")
dbg_embeds = model.encode(list(map(synth_txt, dbg_txt)))
print(dbg_embeds.shape)
for i, label in enumerate(birch.predict(dbg_embeds)):
    print(dbg_txt[i])
    print(synth_txt(dbg_txt[i]))
    print(f"    {clusters[label][0]}")
    print()
sim = model.similarity(dbg_embeds, dbg_embeds)
sim

arud+literal
SYNTACTIC SIMILARITY
(7, 768)
عمر الفتى ذكره لا طول مدته
العلم اسم قام ه لا اسم اسم ه


KeyError: 4567

In [5]:
print(SYNTH_METHOD)

corpus2 = []
with open(f'antar.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if len(line) == 0: continue
        corpus2.append(line)
corpus2[:10]

ud+literal


['هَلْ غادَرَ الشُّعَرَاءُ مِنْ مُتَرَدَّمِ',
 'أَم هَلْ عَرَفْتَ الدَّارَ بَعْدَ تَوَهُّمِ',
 'يَا دَارَ عَبْلَةَ بِالجَوَاءِ تَكَلَّمِي',
 'وَعِمِي صَبَاحاً دَارَ عَبْلَةَ وَاسْلَمِي',
 'فَوَقَفْتُ فِيْهَا نَاقَتِي وَكَأَنَّهَا',
 'فَدَنٌ لِأَقْضِيَ حَاجَةَ المُتَلَوِّمِ',
 'وَتَحُلُّ عَبْلَةُ بِالجَوَاءِ وَأَهْلُنَا',
 'بِالحَزْنِ فَالصَّمَّانِ فَالمُتَثَلَّمِ',
 'حُيِّيْتَ مِنْ طَلَلٍ تَقَادَمَ عَهْدُهُ',
 'أَقْوَى وَأَقْفَرَ بَعْدَ أُمِّ الهَيْثَمِ']

In [6]:
print(SYNTH_METHOD)
corpus2_txfm = list(map(synth_txt, corpus2))
embeddings = model.encode(corpus2_txfm)
print(embeddings.shape)
model.similarity(embeddings[:10,:], embeddings[:10,:])

ud+literal
(148, 768)


tensor([[1.0000, 0.7254, 0.6753, 0.5923, 0.7133, 0.7120, 0.6892, 0.5891, 0.8856,
         0.6990],
        [0.7254, 1.0000, 0.6131, 0.5702, 0.6948, 0.7832, 0.6862, 0.6804, 0.6178,
         0.8267],
        [0.6753, 0.6131, 1.0000, 0.7099, 0.6706, 0.6661, 0.7284, 0.5277, 0.6431,
         0.6898],
        [0.5923, 0.5702, 0.7099, 1.0000, 0.7130, 0.6580, 0.8405, 0.5941, 0.6133,
         0.7792],
        [0.7133, 0.6948, 0.6706, 0.7130, 1.0000, 0.8320, 0.7885, 0.7173, 0.7099,
         0.7781],
        [0.7120, 0.7832, 0.6661, 0.6580, 0.8320, 1.0000, 0.7837, 0.7934, 0.6416,
         0.7747],
        [0.6892, 0.6862, 0.7284, 0.8405, 0.7885, 0.7837, 1.0000, 0.7406, 0.6735,
         0.8553],
        [0.5891, 0.6804, 0.5277, 0.5941, 0.7173, 0.7934, 0.7406, 1.0000, 0.5515,
         0.6944],
        [0.8856, 0.6178, 0.6431, 0.6133, 0.7099, 0.6416, 0.6735, 0.5515, 1.0000,
         0.6905],
        [0.6990, 0.8267, 0.6898, 0.7792, 0.7781, 0.7747, 0.8553, 0.6944, 0.6905,
         1.0000]])

In [7]:
from sklearn.cluster import MeanShift
meanshift = MeanShift(bandwidth=2).fit(embeddings)

In [8]:
clusters = save_clustering("meanshift", meanshift.labels_)

NameError: name 'save_clustering' is not defined